### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Turkish
Lang Id = 2
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/3-Word Group In Youtube Sentence Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
#parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Result/2-Word Group Prepare For Youtube Tool"

In [9]:
#df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28 = df_28.head(200)
df_28

,video_id,start_time,end_time,text
0,IGJVw1HjIHc,4267,4268,yok çok değil o kadar da değil
1,YedJhuZQqpI,6690,6693,hayır hayır bir ama ben sen
2,pvsImT6E4yM,6352,6354,ama bu kadar bir şey yok da
3,FzHO9BqY_Vc,50,52,yok yok ben de bir şey yok
4,4LtEd0U7rOw,297,300,yok bu değil de o da değil
...,...,...,...,...
131,WspUg4WmNHI,232,235,ama ben var
132,LsrRVWm2ivA,3762,3764,mi evet çok
133,cslXqrcH5qg,3896,3897,ne kadar bu
134,eQZssVr1Y64,1301,1308,ben de evet


In [10]:
#df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200 = df_200.head(200)
df_200

,video_id,start_time,end_time,text
0,9Wx3JJTq_Hw,84,88,senin için önemli olan tek şey hala burada bil...
1,ZdeFof81xKs,721,726,bilmiyorum sen ne demek istiyorsun ben de onu ...
2,zgQbOmPp9pk,7920,7926,var da bu bir sorun olur mu benim için ben bun...
3,f8iA6v4mRAY,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...
4,AvfpnsqaWw4,2488,2498,yapıyorsun güzel olur değil tabii ki olur ya n...
...,...,...,...,...
195,_IimozO_x50,2467,2470,size de aynı şekilde çok teşekkürler
196,zIrVQdIcpAA,374,377,evet bu doğru neden böyle yapıyorsun
197,L9bkiAADDIY,4704,4706,var tabii de yani şimdi orada burada
198,XnakBVeh0n4,1891,1893,ya bilmiyorum çok fazla mı ben böyle


In [11]:
#df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000 = df_1000.head(200)
df_1000

,video_id,start_time,end_time,text
0,KsZ_8oIQsv4,319,325,başka biriyle herhangi bir şey yaşıyor musun k...
1,cUAQrHbbpZ8,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...
2,Mh3bgDLKt2o,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...
3,sHnvEDmE3so,273,278,benim burada ne işim var neden bunu yapıyorum ...
4,zgQbOmPp9pk,5381,5389,bu işi yaparım dedim o zaman süper dedi bitti ...
...,...,...,...,...
195,io3ajc3ysBY,1915,1918,sen şey yapma arkadaşım sen de işte kız istemi...
196,kcWDeR30wE4,436,443,geçen sene çok zor geçti ve sanırım herkes içi...
197,iMj5gB0XHRw,4139,4144,bizi rahat bırak kadın rahat bırak ya bugün be...
198,EO4RHGzuFsw,528,531,erken hareket edin diyor ama biz gerçekten mük...


In [12]:
#df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000 = df_5000.head(200)
df_5000

,video_id,start_time,end_time,text
0,MQ_nenSkjbo,380,396,şimdi ben seni tabii hem kişisel olarak hem de...
1,dhV8ToP5VTQ,399,407,ama ben kimseye durup da spor yapmak isteyen i...
2,SgYj1SsG1LQ,1638,1647,daha ciddi bir şey hak ettiğini biliyor ve bun...
3,3mpFwkjir0A,1630,1636,böyle karmaşık birileri hava atıyor biri karı ...
4,GimA7_uhyWg,811,820,ve bir şansın olsa belki de bazı şeyleri daha ...
...,...,...,...,...
195,IlCIjejtda8,1703,1709,hoş durdu böyle böyle bunları düşünüyorsunuz a...
196,PzC8LWAObVw,1780,1786,yani senin gerçekten başka biriyle olduğunu da...
197,_1B7RWV8xXo,235,247,gerçekten senin dostun varmış benim ise sadece...
198,sDR1xwXjMuE,3122,3131,sen dedi dedim dedim ince ağzına daha önce asl...


In [13]:
#df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000 = df_10000.head(200)
df_10000

,video_id,start_time,end_time,text
0,LPrMSJHDDcA,679,690,on tane geliyor haftada üç dört kere şeye çıkı...
1,FIcnpr3UtaQ,695,705,okulun ilk günü ne yapacağımızı bilmiyorduk bi...
2,MQ_nenSkjbo,380,396,şimdi ben seni tabii hem kişisel olarak hem de...
3,ywf4HpQKuf0,109,119,o yüzden aslında hep sosyal medya ile bir şeki...
4,nB0Q5v1_kUs,362,371,ve bu gözle gelecek olan ikinci karakteri ki ö...
...,...,...,...,...
195,9GbYnL0tvcg,3638,3646,ihtiyacım olmadığı için yani zaten şey konuşma...
196,EyQTMdxuqSs,4959,4967,diye mi sorular var ya gelmişsin zaten hani bi...
197,g9hnBqWAXns,100,110,öyle dedik yapacağız şuraya kadar yapalım bura...
198,_hmY0rBynbk,304,309,çok uzun süre değil ve şu anda da gördüğümüz g...


In [14]:
#df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,on tane geliyor haftada üç dört kere şeye çıkı...,679,690,on tane geliyor haftada üç dört kere şeye çıkı...,LPrMSJHDDcA,https://www.youtube.com/watch?v=LPrMSJHDDcA&t=...
1,ancak herhangi bir sorusu varsa ya da belki bi...,682,691,ancak herhangi bir sorusu varsa ya da belki bi...,J0hq13epy2k,https://www.youtube.com/watch?v=J0hq13epy2k&t=...
2,oradaki sadece şu çok önemliydi dört kere geld...,3834,3843,oradaki sadece şu çok önemliydi dört kere geld...,LOQOS5IJtlM,https://www.youtube.com/watch?v=LOQOS5IJtlM&t=...
3,sen hayattan ne istiyorsan onun için çalışmalı...,1342,1377,sen hayattan ne istiyorsan onun için çalışmalı...,luv56Zk_gkY,https://www.youtube.com/watch?v=luv56Zk_gkY&t=...
4,herhalde geldiğim en yoksul ülke burası oldu y...,1074,1115,herhalde geldiğim en yoksul ülke burası oldu y...,9jCA8ZQD2IY,https://www.youtube.com/watch?v=9jCA8ZQD2IY&t=...
...,...,...,...,...,...,...
195,işte o yüzden salata falan yiyin diyor sürekli...,1563,1568,işte o yüzden salata falan yiyin diyor sürekli...,P_Xncte6zqg,https://www.youtube.com/watch?v=P_Xncte6zqg&t=...
196,resimde görüldüğü gibi yaparsınız veya tek ren...,60,67,i ster resimde görüldüğü gibi yaparsınız veya ...,PrlXyxJfn3Q,https://www.youtube.com/watch?v=PrlXyxJfn3Q&t=60s
197,o haftadan sonra geldi çok şükür sanki biraz d...,526,529,tat koku duyum o haftadan sonra geldi çok şükü...,T7LLvxdAQNg,https://www.youtube.com/watch?v=T7LLvxdAQNg&t=...
198,sonra akşam bakıyor yine orada devamlı böyle e...,1807,1813,sonra akşam bakıyor yine orada devamlı böyle e...,3xCLc2owEtI,https://www.youtube.com/watch?v=3xCLc2owEtI&t=...


In [15]:
#df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,baktığınızda aynı fikirde misiniz çoğunlukla h...,410,425,baktığınızda aynı fikirde misiniz çoğunlukla h...,CULrx3yerE8,https://www.youtube.com/watch?v=CULrx3yerE8&t=...
1,on tane geliyor haftada üç dört kere şeye çıkı...,679,690,on tane geliyor haftada üç dört kere şeye çıkı...,LPrMSJHDDcA,https://www.youtube.com/watch?v=LPrMSJHDDcA&t=...
2,yeni yılda eve şans gelmesi için gece yarısınd...,238,251,yeni yılda eve şans gelmesi için gece yarısınd...,aAa2PRgjkcg,https://www.youtube.com/watch?v=aAa2PRgjkcg&t=...
3,bir sürü kızla çıkmış bir çocuk kızları düşünü...,587,595,bir sürü kızla çıkmış bir çocuk kızları düşünü...,bwEEzF905Gk,https://www.youtube.com/watch?v=bwEEzF905Gk&t=...
4,cumartesi bildiğin iki saat aralıksız dört far...,333,345,cumartesi bildiğin iki saat aralıksız dört far...,LPrMSJHDDcA,https://www.youtube.com/watch?v=LPrMSJHDDcA&t=...
...,...,...,...,...,...,...
195,ve üretim merkezleri ardı ardına kapandı eğiti...,115,124,fabrikalar ve üretim merkezleri ardı ardına ka...,5x3CoCfUKbM,https://www.youtube.com/watch?v=5x3CoCfUKbM&t=...
196,bugün için savaş uçağı motorları füzeler ya da...,200,207,bugün için savaş uçağı motorları füzeler ya da...,s40CrgS55J4,https://www.youtube.com/watch?v=s40CrgS55J4&t=...
197,herkes bu yükü taşımaya çalışıyor ama sağlık ç...,96,103,herkes bu yükü taşımaya çalışıyor ama sağlık ç...,3h4pkUO3Gmk,https://www.youtube.com/watch?v=3h4pkUO3Gmk&t=96s
198,para kazanmak istiyorsanız ya da kendinizi tan...,3977,3988,para kazanmak istiyorsanız ya da kendinizi tan...,xiaSoh6_9MI,https://www.youtube.com/watch?v=xiaSoh6_9MI&t=...


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [18]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [19]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [20]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [21]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [22]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [23]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [24]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [25]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [26]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [27]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [28]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [29]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [30]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [31]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [32]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [33]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [34]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [35]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [36]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [37]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [38]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [39]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [40]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [41]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [42]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [43]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [44]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [45]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [46]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [47]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [48]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1674385602
  nanos: 199330000
}